In [22]:
import  os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [23]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} #dictonary to store session ids

#function to return chat message history based on the session id from the store dictonary if it is present in the store. 
def get_session_history(session_id:str)->BaseChatMessageHistory:
    #session_id is used to distingusish one message history from another
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history) #to interact with llm based on message history

In [25]:
config={"configurable":{"session_id":"chat_1"}}

Testing the config and session

In [26]:
with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Walter White")],
    config=config
) 
#to test the config and session

AIMessage(content="Hello, Walter. It's nice to meet you.  \n\nIs there anything I can help you with today? \n\n(Please remember, I'm an AI and can't offer advice on illegal activities.)\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 16, 'total_tokens': 64, 'completion_time': 0.087272727, 'prompt_time': 0.00012737, 'queue_time': 0.01532244, 'total_time': 0.087400097}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-28443f03-d658-48d3-8346-8a39f506414b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 48, 'total_tokens': 64})

In [27]:
with_message_history.invoke(
    [HumanMessage(content="say my name")],
    config=config
) 

#testing the config and session

AIMessage(content="Walter. \n\nIt's good to meet you, Walter. 😊  \n\nWhat can I do for you?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 75, 'total_tokens': 102, 'completion_time': 0.049090909, 'prompt_time': 0.002082842, 'queue_time': 0.013634655, 'total_time': 0.051173751}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b2929e3e-dcf4-44c3-9f38-a210237a3933-0', usage_metadata={'input_tokens': 75, 'output_tokens': 27, 'total_tokens': 102})

In [28]:
config1={"configurable":{"session_id":"chat_2"}}
with_message_history.invoke(
    [HumanMessage(content="say my name")],
    config=config1
) 
#using different session


AIMessage(content='Please tell me your name so I can say it! 😊  </p>\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 12, 'total_tokens': 31, 'completion_time': 0.034545455, 'prompt_time': 8.001e-05, 'queue_time': 0.01506446, 'total_time': 0.034625465}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-da670ea9-2aaa-4765-b6eb-0683975ad7ea-0', usage_metadata={'input_tokens': 12, 'output_tokens': 19, 'total_tokens': 31})

PROMPT TEMPLATES AND CHAT HISTORY

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpul F1 Journalist.Answer the questions about F1 your unbiased point of view."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [30]:
chain.invoke({"messages":[HumanMessage(content="what to expect from upcoming lasvegas gp?")]})

AIMessage(content="The upcoming Las Vegas Grand Prix is shaping up to be a spectacle unlike any other in Formula 1 history.  Here's what we can expect, from my unbiased journalist perspective:\n\n**The Track:**\n\n* **High-speed thrills:** The circuit weaves through the heart of Las Vegas, featuring long straights perfect for overtaking and showcasing the cars' top speeds. Expect to see some seriously impressive speeds, potentially even exceeding 200 mph in certain sections.\n* **Night racing drama:** Holding the race under the glittering lights of the Las Vegas strip adds a unique layer of excitement. The lights, the city's buzz, and the potential for unexpected twists under the cover of darkness promises a truly unforgettable atmosphere.\n* **Technical challenges:** Despite the focus on speed, the circuit also incorporates tight corners and chicanes, requiring drivers to be precise and adaptable. This blend of high-speed sections and technical challenges will test the drivers' skills

Prompt template with chat history

In [31]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="what to expect from upcoming lasvegas gp?")],
    config=config
)
response.content

"The Las Vegas Grand Prix is shaping up to be one of the most exciting races on the F1 calendar, and here's why:\n\n**High-Speed Thrills:**\n\n* The track itself is a street circuit with high-speed corners and long straights, reminiscent of Monaco's glamour but with more overtaking opportunities.  Expect drivers to push their cars to the limit, potentially setting new speed records.\n\n**Night Race Spectacle:**\n\n*  Racing under the glittering lights of the Las Vegas Strip will be an unprecedented spectacle. The city's iconic skyline will provide a stunning backdrop, and the nightlife atmosphere should add to the excitement.\n\n*  This night race format will also present unique challenges for teams, especially regarding tire management and adapting to the changing track conditions as temperatures drop.\n\n**Close Quarter Battles:**\n\n* The narrow street circuit layout is expected to lead to close and intense battles between drivers. Overtaking will be a key factor, and we could see s

In [32]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpul F1 Journalist.Answer the questions about F1 from your unbiased point of view in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [33]:
response=chain.invoke({"messages":[HumanMessage(content="what to expect from upcoming lasvegas gp?")],"language":"French"})
response.content

"##  Le Grand Prix de Las Vegas: Des attentes à la hauteur des lumières de la Strip ?\n\nLe Grand Prix de Las Vegas, ce nouvel événement flamboyant sur le calendrier F1, promet une expérience unique pour les pilotes et les fans. \n\n**Sur le plan sportif:**\n\n* **La piste:** Le circuit urbain, éclairé par les néons de la Strip, offre des virages rapides et serrés, des lignes droites longues et des défis techniques importants. On peut s'attendre à des courses spectaculaires, avec des dépassements audacieux et des stratégies offensives.\n* **Les favoris:** Les équipes et pilotes habituels des avant-postes, comme Red Bull, Ferrari et Mercedes, devraient être en mesure de briller sur ce circuit. Cependant, le caractère unique du circuit pourrait offrir une opportunité aux équipes moins performantes de surprendre.\n* **Les risques:** Le circuit urbain, avec ses nombreux virages et sa vitesse élevée, présente des risques importants. Les pilotes devront être extrêmement concentrés pour évite

In [34]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [35]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {
        'messages': [HumanMessage(content="what to expect from upcoming lasvegas gp?")],"language":"French"
    },
    config=config
)
response.content

"##  Le Grand Prix de Las Vegas : Des attentes élevées pour un événement unique\n\nLe Grand Prix de Las Vegas arrive à grands pas, et l'excitation est palpable ! Cette course, qui marque l'arrivée d'un nouveau circuit urbain sur le calendrier de la Formule 1, promet d'être une expérience exceptionnelle pour les pilotes et les fans. \n\nVoici ce que nous pouvons attendre de ce GP :\n\n* **Un spectacle visuel époustouflant:**  Le circuit de Las Vegas, traversant les rues emblématiques de la ville, promet des décors spectaculaires et des lumières scintillantes. La course se déroulera la nuit, offrant un spectacle unique sous les néons de la ville.\n\n* **Des vitesses élevées et des duels serrés:**  Le circuit est long et rapide, avec des virages serrés et des sections droites longues.  Cela devrait permettre aux pilotes de pousser leurs voitures à leurs limites et de livrer des duels palpitants. \n\n* **Un défi pour les pilotes:**  La conduite dans un circuit urbain exige une grande conce

In [36]:
response=with_message_history.invoke(
    {
        'messages': [HumanMessage(content="is it happening on sunday or saturday")],"language":"English"
    },
    config=config
)
response.content

"The Las Vegas GP is scheduled for **Saturday, November 18th, 2023**.  \n\nIt's a bit unusual to have a Formula 1 race on a Saturday, but the unique nature of this event and the Vegas schedule makes it happen! \n"

In [37]:
response=with_message_history.invoke(
    {
        'messages': [HumanMessage(content="so italways happens on saturday?")],"language":"English"
    },
    config=config
)
response.content

'You are right to point that out! \n\nThe Las Vegas GP is a **one-off** event happening on a Saturday this year.  \n\nNormally, Formula 1 races are held on Sundays.  \n'

MANAGING CHAT HISTORY WITH LANGCHAIN

In [38]:
from langchain_core.messages import SystemMessage,trim_messages
### trim_messages is used to reduce how many messages we're sending to the model. it helps us to specify how many tokens we want to keep 
# along with other parameters like if we want to always keep the system message and whether to allow partial messages
trimmer = trim_messages(
    max_token = 100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

CHAT HISTORY AND RETRIEVERS

In [39]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(model="Llama3-8b-8192",groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x303abe3b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x303ad49d0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")#386 dimensions